#Market Microstructure Student Notebook
In this notebook you will find the accompanying exercises to today's market microstructure lecture. The aim of this session is to investigate how meta order size affects the success of the TWAP (time weighted average price) trade execution algorithm and discuss various improvements to the standard TWAP algorithm. Both of these will be discussed in the lecture and the task involves using the limit order book simulation data, produced by the code below, to quantify execution slippage as a function of meta-order size.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Creation of the Limit Order Book Simulation as a Multi-indexed DataFrame.
The code below creates the dataframe we will be using to model a limit order book. The idea is that the trading period is split into 10000 intervals and during each of these intervals the available limit orders are stored. Sub- columns store the price limit and volume of all bid and ask orders currently in the LOB. The index then corresponds to each individual limit order. Once the code block below has executed don't run it again as it takes a fairly long time to run and also it will change the resultant dataframe as it relies on random number generation.

In [2]:
# just run all this cell
def volume_curve(t):
  return (t/750)**2*np.exp((t/500)-2) + np.cos(t/500+0.01)

def p_list(t):
  price_list = list(set([round(1/3*(t+2+np.random.normal(0,2))**(-1) +
                               np.random.normal(0,.01) +
                               np.random.pareto(0.5),2) for j in range(np.random.randint(1000,1500))]))
  return np.sort(price_list)

ABM = pd.Series([100] + list(100 + 1/20*pd.Series(np.random.normal(scale = 1, size = 1000-1)).cumsum()))
price_list = [p_list(t) for t in np.arange(0,1000,1)]

bid_price = []
bid_volume = []
ask_price = []
ask_volume = []
for p in range(len(price_list)):
  k = np.random.randint(0,5)
  i = np.random.randint(40,80)
  j = np.random.randint(40,80)
  bid_volume.append(100*np.random.randint(10,100,i))
  bid_price.append(price_list[p][k:i+k]+ABM[p])
  ask_price.append(price_list[p][k+i:k+i+j] + ABM[p])
  ask_volume.append(100*np.random.randint(10,100,j))

LOB_index = pd.MultiIndex.from_tuples((x,y) for x in range(0,1000) for y in ["bid_price","bid_volume","ask_price","ask_volume"])
df = pd.DataFrame(index = LOB_index, columns = range(500))
for i in range(1000):
  df.loc[(i, "bid_price"), range(len(bid_price[i]))] = np.round(bid_price[i][::-1], 2)
  df.loc[(i, "bid_volume"), range(len(bid_volume[i]))] = np.round(10*bid_volume[i]*volume_curve(i), 0)
  df.loc[(i, "ask_price"), range(len(ask_price[i]))] = np.round(ask_price[i], 2)
  df.loc[(i, "ask_volume"), range(len(ask_volume[i]))] = np.round(10*ask_volume[i]*volume_curve(i), 0)

limit_order_book = df.T

In [3]:
limit_order_book.head()

0                                         1                         \
  bid_price bid_volume ask_price ask_volume bid_price bid_volume ask_price   
0     100.3    49998.0    100.31    38998.0    100.24     9999.0    100.25   
1    100.29    33998.0    100.32    87996.0    100.23    27998.0    100.26   
2    100.28    14999.0    100.33    25999.0    100.22    62995.0    100.27   
3    100.27    10000.0    100.34    35998.0    100.21    52996.0    100.28   
4    100.26    74996.0    100.35    63997.0     100.2    17999.0    100.29   

                   2               ...       997                  998  \
  ask_volume bid_price bid_volume  ... ask_price ask_volume bid_price   
0    94993.0    100.51    46995.0  ...     98.38    25399.0     98.48   
1    19999.0     100.5    52995.0  ...     98.39    33419.0     98.47   
2    42997.0    100.49    76993.0  ...      98.4    94911.0     98.45   
3    50996.0    100.48    56994.0  ...     98.41    74859.0     98.44   
4    34997.0    100.47    62994.0  ...     98.42    49461.0     98.43   

                                        999                                  
  bid_volume ask_price ask_volume bid_price bid_volume ask_price ask_volume  
0    56364.0     98.49    73811.0     98.45    16167.0     98.46    13473.0  
1   107361.0      98.5    63074.0     98.44    63322.0     98.47   130685.0  
2    68443.0     98.51    26840.0     98.43   117213.0     98.48    18862.0  
3    96625.0     98.52    22814.0     98.42    74100.0     98.49    16167.0  
4   100651.0     98.53    77837.0     98.41   126643.0      98.5    40418.0  

[5 rows x 4000 columns]

# Exercise 1
First lets visualise the data by plotting how the mid price varies with time.

## Exercise 2
The task here is to complete the function below that for a given buy order of lot size n and meta order size q determine the average price paid for the order. Then once this has been written produce a graph of different q against the average order price for fixed n. Also it is worth considering what behaviour the mid price exhibits and how this affects order size q.

In [ ]:
def TWAP_execution_algorithm(n,q):
  pass

## Extension
Think about how the code might be adapted for VWAP and PoV strategies and also what are some of potential limitations in how we are modelling the LOB.

The code above for the TWAP strategy can be adjusted to from a VWAP strategy by simply including a weight function of the time index which in the solution above is i. The weight function that we will use is the volume_curve function defined above. For a percent of volume calculation there is only one input parameter (the precent of volume) so at each time index this percent of all total volume mut be calculated and then used to determine how many limit orders are filled at each timestep.

The main drawback of such a limit order book model is that it is static. Usually market orders coming through can then trigger more orders or lead to a reduction in future orders. As the limit order book is fixed while the algorithms are running we do not model this. Modelling these effects is complex although mathematical models have been developed to model trade propagation effects. For more on this check out the book trades, quotes and prices: financial markets under the microscope by Martin Gould and others.